# LFortran简介

## 什么是 lfortran？

lfortran 是一个现代的 Fortran 编译器基础设施，设计用于提供高级功能和强大的编译器工具。它的目标是成为一个支持现代 Fortran 标准的开源编译器，同时提供灵活的编程接口，使得编译器开发者和用户能够方便地操作和扩展 Fortran 代码。

## 主要功能

1. **抽象语法树 (AST)**：lfortran 提供了对 Fortran 代码的详细抽象语法树表示，使用户能够轻松分析和修改代码结构。这对于理解和优化代码非常重要。
2. **即时编译 (JIT)**：lfortran 支持即时编译技术，可以在运行时编译和执行 Fortran 代码，这对于科学计算和快速原型开发非常有用。
3. **交互式使用**：通过与 Jupyter Notebook 的集成，lfortran 允许用户在交互式环境中编写和执行 Fortran 代码，增强了教学和研究的便利性。
4. **高性能计算**：lfortran 旨在支持高性能计算的需求，能够生成高效的机器代码，适用于数值模拟和科学计算等领域。

## 重点

lfortran 是一个强大的工具，为 Fortran 编程带来了现代化的编译器技术。通过对 AST 的支持和与 Jupyter Notebook 的集成，lfortran 不仅适合科学计算和高性能计算的需求，也为教学和研究提供了极大的便利。

此外，lfortran 使用 LLVM（Low Level Virtual Machine）作为其后端编译器基础设施。LLVM 是一个高度优化的编译器框架，能够生成高效的机器代码，并支持多种硬件架构。通过使用 LLVM，lfortran 能够利用其强大的优化功能和跨平台支持，为用户提供高性能和灵活的编译体验。

让我们先从一些Fortran的例子出发，进行一下交互式体验。

## 计算

In [1]:
1+2+3

6

In [2]:
(2+3)*3

15

## 全局变量

In [1]:
integer :: i

In [2]:
integer :: j

In [3]:
i = 5
j = 6

In [4]:
i

5

In [5]:
(i+j)*3

33

## 函数
你可以声明一个函数并在表达式中使用它：

In [6]:
integer function fn(a, b)
integer, intent(in) :: a, b
fn = a + b
end function

In [7]:
fn(2, 3)

5

In [8]:
fn(2, 3)*3

15

## 控制流与打印
你可以使用循环、if 语句或 `print` 函数。

In [11]:
integer :: k
do k = 1, 4
    if (k == 3) cycle
    print *, "variable i =", k
end do

variable i = 1
variable i = 2
variable i = 4


## 魔法命令


### 显示抽象语法树
在解析后显示抽象语法树 (AST)（仅基于语法，无语义）：

In [12]:
integer :: n
n = 5
j = 0

In [13]:
%%showast
do i = 1, n
    j = j + i
end do

(TranslationUnit
    [(DoLoop
        0
        ()
        0
        i
        1
        n
        ()
        [(Assignment
            0
            j
            (+ j i)
            ()
        )]
        ()
        ()
    )]
)

### 显示 LLVM 代码

In [14]:
%%showllvm
do i = 1, n
    j = j + i
end do

; ModuleID = 'LFortran'
source_filename = "LFortran"

@i = external global i32
@j = external global i32
@k = external global i32
@n = external global i32

define void @__lfortran_evaluate_13() {
.entry:
  store i32 0, i32* @i, align 4
  br label %loop.head

loop.head:                                        ; preds = %loop.body, %.entry
  %0 = load i32, i32* @i, align 4
  %1 = add i32 %0, 1
  %2 = load i32, i32* @n, align 4
  %3 = icmp sle i32 %1, %2
  br i1 %3, label %loop.body, label %loop.end

loop.body:                                        ; preds = %loop.head
  %4 = load i32, i32* @i, align 4
  %5 = add i32 %4, 1
  store i32 %5, i32* @i, align 4
  %6 = load i32, i32* @j, align 4
  %7 = load i32, i32* @i, align 4
  %8 = add i32 %6, %7
  store i32 %8, i32* @j, align 4
  br label %loop.head

loop.end:                                         ; preds = %loop.head
  br label %return

return:                                           ; preds = %loop.end
  ret void
}

declare i32 @fn(i32

# 显示汇编代码

In [15]:
%%showasm
do i = 1, n
    j = j + i
end do

	.text
	.file	"LFortran"
	.globl	__lfortran_evaluate_14
	.p2align	4, 0x90
	.type	__lfortran_evaluate_14,@function
__lfortran_evaluate_14:
	.cfi_startproc
	movq	i@GOTPCREL(%rip), %rax
	movl	$0, (%rax)
	movq	n@GOTPCREL(%rip), %rcx
	movq	j@GOTPCREL(%rip), %rdx
	.p2align	4, 0x90
.LBB0_1:
	movl	(%rax), %esi
	incl	%esi
	cmpl	(%rcx), %esi
	jg	.LBB0_3
	movl	(%rax), %esi
	incl	%esi
	movl	%esi, (%rax)
	addl	%esi, (%rdx)
	jmp	.LBB0_1
.LBB0_3:
	retq
.Lfunc_end0:
	.size	__lfortran_evaluate_14, .Lfunc_end0-__lfortran_evaluate_14
	.cfi_endproc

	.section	".note.GNU-stack","",@progbits
